In [ ]:
import mqr
from mqr.nbtools import vstack
from mqr.plot import Figure

In [ ]:
# Data and calculation libraries
import numpy as np
import pandas as pd
import scipy.stats as st
import statsmodels.formula.api as smf
import statsmodels.api as sm

---
# DoE

The `mqr` library provides a wrapper around pyDOE3 that makes the designs easy to combine and then use in a DataFrame for saving, loading with observations/experimental results.

The features are:
* creation from pyDOE3 functions fullfact, fracfact and ccdesign,
* labelling with point types, for easy management in analysis,
* transforming from labels to physical values, for easier experimental technique,
* easy concatenation and blocking.

The main type is `mqr.doe.Design`.  
To concatenate designs use a plus symbol: `design1 + design2`.  
To block designs, either pass the block number when creating the design (`Design.from_fullfact(..., block=1)`),
or change the block for a design using `design.as_block(...)`.  
To randomise a design, call `design.randomise_runs()`, which preserves blocks by default.  
To scale labels to physical values, define a `mqr.doe.Transform` and apply it to a design like matrix multiplication `Design @ Transform`.

---
## Experimental workflow
1. Design the experiment using the tools in mqr.doe (or pyDOE3 directly).
1. Randomise the runs.
1. Save the design to a _design file_, and append the experimental observations as a new column to make an _experiment file_.
1. (optional) Instead of creating a new file, get the dataframe version of a design and enter data directly into the notebook as an extra column on the dataframe: `design['Observation'] = np.array([...])`
1. Load the _experiment file_ (if you created one) ready for analysis with ANOVA and regression tools.

---

In [ ]:
from mqr.doe import Design

### 1 Fractional Factorial Design

In [ ]:
names = var_list = ['x1', 'x2', 'x3', 'x4', 'x5', 'x6']
gen = 'a b c d abcd abc'

Design.from_fracfact(names, gen)

### 2 Fractional Factorial with Centre Points

In [ ]:
names = ['x1', 'x2', 'x3', 'x4', 'x5', 'x6']
gen = 'a b c d abcd abc'
nc = 3

Design.from_fracfact(names, gen) + Design.from_centrepoints(names, nc)

### 3 Central Composite Design — Full Factorial
With blocking

In [ ]:
names = ['x1', 'x2', 'x3', 'x4']
levels = [2, 2, 2, 2]
nc = 3

blk1 = Design.from_fullfact(names, levels) + Design.from_centrepoints(names, nc)
blk2 = Design.from_axial(names) + Design.from_centrepoints(names, nc)
design = blk1.as_block(1) + blk2.as_block(2)
design

### 4 Central Composite Design — Fractional Factorial
With blocking

In [ ]:
names = ['x1', 'x2', 'x3', 'x4']
gen = 'a b c abc'
nc = 3

blk1 = Design.from_fracfact(names, gen) + Design.from_centrepoints(names, nc)
blk2 = Design.from_axial(names) + Design.from_centrepoints(names, nc)
design = blk1.as_block(1) + blk2.as_block(2)
design

---
# Practicalities

### 5 Replicating the runs
The optional argument `label` adds a column that labels replicates.

In [ ]:
blk1.replicate(2, label='Rep')

### 6 Randomising the runs
Rearrange the rows of a dataframe by calling `design.randomise_runs()`.
Blocks or factor levels can be kept in order by including the name of the block or factor in the list `order`.
This example keeps blocks in order, and also `x1` is ordered per block
(`block` is specified before `x1` in the ordering list to give it priority).
The remaining factors `x2`, `x3` and `x4` are randomised within blocks and `x1` levels.

In [ ]:
np.random.seed(1234) # Warning: seeding the random number generator will produce the same ordering every run
design = blk1.as_block(1) + blk2.as_block(2) + blk1.as_block(3) + blk2.as_block(4)
# design.randomise_runs() # this would completely randomise all runs
design.randomise_runs('Block', 'x1')

### 7 Transforming the level labels to physical values
Writing down exactly which values correspond to each level is convenient for careful experimental technique.
These values can be read off a screen or printed while conducting an actual experiment.

First, define a transform that maps the levels that correspond to each label
(when `mqr.doe` constructs a transform from labels like below, it assumes the transform is affine).

The transforms can be callable objects like a lambda, function or `mqr.doe.Transorm`.
Or they can be dicts that give a mapped value for every coded level.

This example transforms `x1` with an affine transform, `x2` with an affine transform that is inversely proportional to the coded variable and `x4` with a categorical value. The factor `x3` is left in coded units.

In [ ]:
from mqr.doe import Transform

design = mqr.doe.Design.from_fullfact(names, levels)

transforms = {
    'x1': Transform.from_map({-1:100, 1:110}),
    'x2': lambda x: -x + 5,
    'x4': {-1: 'low', 1: 'high'},
}

design.transform(**transforms)

### 8 Save the experimental design to file for printing etc.
The `index_label` argument in `DataFrame.to_csv(...)` tells Pandas to include the index column with the given name.

In [ ]:
# np.random.randint(0, 2**32-1)

np.random.seed(1294194915) # Randomly generated seed (above)
frozen_design = design.randomise_runs().to_df()
frozen_design.to_csv(
    'doe-section6-1294194915.csv',
    index_label='run')
frozen_design.to_excel(
    'doe-section6-1294194915.xlsx',
    index_label='run')